In [ ]:
import pandas as pd
import tensorflow as tf
import keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_val = X_train_full[-10000:]
y_val = X_train_full[-10000:]
X_train = X_train_full[:-10000]
y_train = y_train_full[:-10000]

# Optimizer

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

# Early stopping

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, monitor="val_accuracy", min_delta= 0.01, restore_best_weights=True)

# Checkpoint

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("model.tf", save_best_only=True, monitor="val_accuracy", mode="max")

# Normaliseren

In [8]:
norm_layer = keras.layers.Normalization()
norm_layer.adapt(X_train)

# MLP model

- **Binaire classificatie**:
  - Loss: binary_crossentropy
  - Activatie: sigmoid (output)
  - Optimizer: adam

- **Meerklassen classificatie**:
  - Loss: categorical_crossentropy (of sparse_categorical_crossentropy)
  - Activatie: softmax (output)
  - Optimizer: adam

- **Regressie**:
  - Loss: mean_squared_error of mean_absolute_error
  - Activatie: Geen (lineaire output)
  - Optimizer: adam of sgd


### volledig model

In [ ]:
def get_model(norm_layer):
  model = keras.Sequential()
  model.add(keras.Input(shape=(10000,))) # aantal features
  if norm_layer is not None:
    model.add(norm_layer)
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(300,activation="relu")) # 300 = aantal hidden (gegeven)
  model.add(keras.layers.Dense(10,activation="softmax"))
  return model

### multiclass

In [ ]:
def get_model():
  model = keras.Sequential()
  model.add(keras.Input(shape=X_train.shape[1:]))
  model.add(keras.layers.Dense(300,activation="relu"))
  model.add(keras.layers.Dense(10,activation="softmax"))
  return model

### binaire classificatie

In [ ]:
def get_model():
  model = keras.Sequential()
  model.add(keras.Input(shape=X_train.shape[1:]))
  model.add(keras.layers.Dense(300,activation="relu"))
  model.add(keras.layers.Dense(1,activation="sigmoid"))
  return model

### regressie

In [ ]:
def get_model():
  model = keras.Sequential()
  model.add(keras.Input(shape=X_train.shape[1:]))
  model.add(keras.layers.Dense(300,activation="relu"))
  model.add(keras.layers.Dense(1))
  return model

# Model summary

In [ ]:
model = get_model()
model.summary()

# Compile model

In [17]:
model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(optimizer=optimizer, loss="mse", metrics=["RootMeanSquaredError"])

# Model trainen

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, batch_size=256, callbacks=[checkpoint_cb, early_stopping_cb])

# Laad beste model in

In [ ]:
best_model = keras.models.load_model("model.tf")


# Model evalueren

In [ ]:
model.evaluate(X_test, y_test)

# Predictions maken

In [ ]:
X_new = X_test[:3]  # pretend these are new instances
y_proba = model.predict(X_new)
y_proba.round(2)
y_pred = y_proba.argmax(axis=-1)